In [1]:
%pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 1.5 MB/s eta 0:00:0000:0100:02m

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import qutip as qt
from qutip import Bloch, QobjEvo, basis, sesolve, sigmay, sigmaz
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
from scipy.constants import h, e

# Operadores de Pauli en el qubit k
def SX(N, k):
    """Create a sigma_X operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N 
    L[k] = qt.sigmax()
    return qt.tensor(L)

def SY(N, k):
    """Create a sigma_Y operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N
    L[k] = qt.sigmay()
    return qt.tensor(L)

def SZ(N, k):
    """Create a sigma_Z operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N
    L[k] = qt.sigmaz()
    return qt.tensor(L)


from qutip import create, destroy

def Sm(N, k):
    """Create a sigma_X operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N 
    L[k] = destroy(2)
    return qt.tensor(L)

def Sp(N, k):
    """Create a sigma_X operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N 
    L[k] = create(2)
    return qt.tensor(L)



In [16]:
# Operador U local en el qubit k
def U_qubit(N, k, params):
   
    theta, phi, lambd = params  # Extrae los 3 parámetros para U

    # Define la matriz de U
    U = np.array([
        [np.cos(theta/2), -np.exp(1j * lambd) * np.sin(theta/2)],
        [np.exp(1j * phi) * np.sin(theta/2), np.exp(1j * (phi + lambd)) * np.cos(theta/2)]
    ])

    # Convierte a Qobj de Qutip
    U_qobj = qt.Qobj(U)

    # Construye el operador tensorial en el sistema de N qubits
    L = [qt.qeye(2)] * N  # Lista con Identidades en todos los sitios
    L[k] = U_qobj  # Sustituye en el qubit k
    return qt.tensor(L)




In [3]:
# Número de qubits
N = 4  

# Vector de parámetros (12 elementos en total: 3 por cada qubit)
params_vec = [np.pi/4, np.pi/3, np.pi/6,  # Qubit 0
              np.pi/2, np.pi/4, np.pi/8,  # Qubit 1
              np.pi/3, np.pi/6, np.pi/2,  # Qubit 2
              np.pi/8, np.pi/3, np.pi/4]  # Qubit 3

# Generar los operadores U para cada qubit
U_0 = U_qubit(N, 0, params_vec[0:3])  # U en el qubit 0
U_1 = U_qubit(N, 1, params_vec[3:6])  # U en el qubit 1
U_2 = U_qubit(N, 2, params_vec[6:9])  # U en el qubit 2
U_3 = U_qubit(N, 3, params_vec[9:12]) # U en el qubit 3

# Comprobamos los operadores generados
print(U_0)
#print(U_1)
#print(U_2)
#print(U_3)


Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[ 0.92387953+0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j         -0.33141357-0.19134172j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.92387953+0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
  -0.33141357-0.19134172j  0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.92387953+0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.      

In [17]:
# Función de coste para el VQE simulado
def Cost(params, *args):
    U_0 = U_qubit(N, 0, params[0:3])  # U en el qubit 0
    U_1 = U_qubit(N, 1, params[3:6])  # U en el qubit 1
    U_2 = U_qubit(N, 2, params[6:9])  # U en el qubit 2
    U_3 = U_qubit(N, 3, params[9:12])
    psi_e = args[0]
    H = args[1]

    Psi = U_3*U_2*U_1*U_0*psi_e
    
    cost = qt.expect(H, Psi)
    return cost
    

In [19]:
N = 4
s0 = qt.basis(2, 0)
s1 = qt.basis(2, 1)

#ESTADOS DE CADA CLASE DE ENTRELAZAMIENTO
psi_C9 = 1/2*(qt.tensor(s0,s0,s0,s0)+qt.tensor(s0,s1,s0,s1)+qt.tensor(s1,s0,s1,s0) + qt.tensor(s1,s1,s1,s1) )
psi_C3 = 1/np.sqrt(2)*(qt.tensor(s0,s0,s0,s0)+ qt.tensor(s1,s0,s1,s0))
psi_C1 = qt.tensor(s0,s0,s0,s0)
psi_C11 = 1/np.sqrt(3)*(qt.tensor(s0,s0,s1,s0)+qt.tensor(s0,s1,s0,s0)+qt.tensor(s1,s0,s0,s0))
psi_C15 = 1/np.sqrt(2)*(qt.tensor(s0,s0,s0,s0)+ qt.tensor(s1,s1,s1,s0))
psi_C19 = 1/2*(qt.tensor(s1,s0,s0,s0)+qt.tensor(s0,s1,s0,s0)+qt.tensor(s0,s0,s1,s0)+ qt.tensor(s0,s0,s0,s1))
psi_C26 = 1/np.sqrt(2)*(qt.tensor(s0,s0,s0,s0)+qt.tensor(s1,s1,s1,s1))
psi_C48 = 1/np.sqrt(8)*(qt.tensor(s0,s0,s0,s0)+qt.tensor(s0,s0,s1,s1)+qt.tensor(s0,s1,s0,s1)+qt.tensor(s0,s1,s1,s0)+qt.tensor(s1,s0,s0,s0)-qt.tensor(s1,s0,s1,s1)+qt.tensor(s1,s1,s0,s1)-qt.tensor(s1,s1,s1,s0))

# Hamiltoniano
def construct_hamiltonian(N, Ops, coeffs):
    """
    Construct the Hamiltonian given the operators and coefficients.

    Args:
        N: Number of qubits in the system.
        Ops: List of operator strings (e.g., 'IIIZ', 'XXYY').
        coeffs: List of coefficients corresponding to the operators.

    Returns:
        Hamiltonian as a qutip.Qobj object.
    """
    I = np.eye(2**(N))
    I = qt.Qobj(I) #Identity in the full Hilbert space
    I.dims = [[2]*(N), [2]*(N)]
    H = 0 * I  # Initialize Hamiltonian as zero operator

    for op, coeff in zip(Ops, coeffs):
        term = I  # Start with identity operator

        for i, char in enumerate(op):
            if char == 'X':
                term = term * SX(N, i)
            elif char == 'Y':
                term = term * SY(N, i)
            elif char == 'Z':
                term = term * SZ(N, i)

        H += coeff * term  # Add the term with its coefficient

    return H
# Convertimos a elementos para qutip sacados de qiskit. Mapeo: Jordan Wigner

#Molecula H2 a r = 0.741
Ops = ['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'] #aqui se cambia

# Extraer los coeficientes
coeffs = [-0.09835117+0.j,  0.17125916+0.j, -0.22297019+0.j,  0.17125916+0.j,
 -0.22297019+0.j,  0.12056779+0.j,  0.16864127+0.j,  0.1658844 +0.j,
  0.0453166 +0.j,  0.0453166 +0.j,  0.0453166 +0.j,  0.0453166 +0.j,
  0.1658844 +0.j,  0.17436685+0.j,  0.12056779+0.j] #aqui igual

# Hamiltoniano para el VQE simulado
Hf = construct_hamiltonian(N, Ops, coeffs)

In [24]:
import numpy as np
from scipy.optimize import minimize

# Definir los límites de los parámetros manualmente para COBYLA
constraints = []
for i in range(12):
    lower_bound = 0
    upper_bound = np.pi if i % 3 == 0 else 2 * np.pi  # Cada tercer parámetro tiene límite en pi
    constraints.append({'type': 'ineq', 'fun': lambda x, i=i: x[i] - lower_bound})  # x[i] >= lower_bound
    constraints.append({'type': 'ineq', 'fun': lambda x, i=i: upper_bound - x[i]})  # x[i] <= upper_bound

# Inicialización de los parámetros aleatorios dentro de los límites
x = [np.random.uniform(0, np.pi) if i % 3 == 0 else np.random.uniform(0, 2*np.pi) for i in range(12)]

# Argumentos de la función de costo
args1 = (psi_C3, Hf) #(Estado de la clase de entrelazamiento, Hamiltoniano de la molecula)

# Minimización usando COBYLA
VQE = minimize(Cost, x, args=args1, method='COBYLA', constraints=constraints, options={'maxiter': 1500, 'disp': True})

print(VQE)


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.45511627629656654
       x: [ 1.937e+00  2.635e+00  4.513e+00  3.784e+00  5.988e+00
            9.105e-01  1.205e+00  5.276e+00  4.911e+00  2.498e+00
            2.345e+00  1.325e-01]
    nfev: 481
   maxcv: 0.0
